In [1]:
import nltk
import re
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# # from nltk.tokenize import word_tokenize
# nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
def get_story_words_relevance(story):
    num_unique_words = 0
    story_unique_words_rel = {} 
    for i in story:
        if not i in story_unique_words_rel:
            story_unique_words_rel[i] = 1
            num_unique_words += 1
        else:
            story_unique_words_rel[i] += 1
    for word in story_unique_words_rel:
        story_unique_words_rel[word] /= num_unique_words
    return [story_unique_words_rel, num_unique_words]

In [3]:
def process_story():
    f = open("story.txt", "r")
    story = f.read()

    story = re.sub(r'[^\w\s]', '', story)
    stop_words = set(stopwords.words('english'))
    story = story.lower()
    word_tokens = word_tokenize(story)

    filtered_story = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_story = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_story.append(w)

    f.close()
    return get_story_words_relevance(filtered_story)

In [4]:
def get_question_relevance(story_words_relevance, question, story_unique_words):
    """Calculating the relevance of question by first calculating sum of 
        relevance of each word in the question. Then we check what range out of the decided ranges(see readme), this falls into"""
    relevance = 0
    for word in question:
        if word in story_words_relevance:
            relevance += story_words_relevance[word]
            
    if relevance <= (0.0005*story_unique_words):
        return 1
    elif relevance > (0.0005*story_unique_words) and relevance <= (0.001*story_unique_words):
        return 3
    elif relevance > (0.001*story_unique_words) and relevance <= (0.002*story_unique_words):
        return 4
    elif relevance > (0.002*story_unique_words) and relevance <= (0.003*story_unique_words):
        return 5
    elif relevance > (0.003*story_unique_words) and relevance <= (0.004*story_unique_words):
        return 6
    elif relevance > (0.004*story_unique_words) and relevance <= (0.005*story_unique_words):
        return 7
    else:
        return 8

In [5]:

def p(u_questions, params):
    for i in range(0, len(u_questions)):
        question_a = u_questions[i]
        if("?\n" in question_a) or ("?" in question_a):
            first_words_lst = [["where", "when", "which", "who"],["what", "do", "does", "would", "could", "should"],["why", "to what extent", "how"]]
            if(".\n" in question_a) or ("." in question_a):
                found = False
                for i in range(0, len(first_words_lst)):
                    first_words = first_words_lst[i]
                    question_a = [word.replace('.\n', '.') for word in question_a]
                    first_word = question_a[question_a.index(".")+1]
                    if ((len(list(filter(lambda word: (word == (first_word.lower())), first_words))))>0):
                        params.append((2*(i+1)))
                        found = True
                        break
                if (not(found)):
                    params.append(0)
                        
            else:
                found = False
                for i in range(0, len(first_words_lst)):
                    first_words = first_words_lst[i]
                    if ((len(list(filter(lambda word: (word == (question_a[0].lower())), first_words))))>0):
                        params.append((2*(i+1)))
                        found = True
                        break
                if (not(found)):
                    params.append(0)

        else:
            cmd_terms_lst = [["calculate", "classify", "formulate", "integrate", "measure", "plot", "select"], 
                         ["define", "determine", "differentiate", "estimate", "find", "label", "recall", "state", "list", "identify", "predict"], 
                         ["annotate", "comment", "deduce", "derive", "distinguish", "find", "otherwise", "prove", "solve", "suggest"], 
                         ["apply", "draw", "describe", "distinguish", "trace", "outline", "use", "verify"], 
                        ["construct", "demonstrate", "explore", "justify"], 
                        ["compare", "contrast", "critique", "deduce", "explain", "interpret"],
                        ["examine", "investigate", "synthesize"], 
                        ["analyze", "discuss", "evaluate", "justify", "create", "design"]]
            found = False
            for word in question_a:
                for i in range(0, len(cmd_terms_lst)):
                    cmd_terms = cmd_terms_lst[i]
                    if word in cmd_terms:
                        params.append((i+1))
                        found = True
                        break
                if found:
                    break
            if (not(found)):
                params.append(0)
    return params


def get_open_ended_nature():
    u_questions = []
    params = []
    with open("questions.txt") as file:
        for line in file:
            temp = str(line.lower())
            u_question = temp.replace("?", " ?")
            u_question = u_question.replace(".", " .").split(' ')
            u_questions.append(u_question)
    
    file.close()
    
    return p(u_questions, params)

In [6]:
def process_questions(story_words_relevance, story_unique_words):
    questions = []
    params = {"relevance": [], "open_ended": []} # index+1 represents question number in the list
    with open("questions.txt") as file:
        for line in file:
            questions.append(line)
    file.close()
    for question in questions:
        stop_words = set(stopwords.words('english'))
        question = question.lower()
        word_tokens = word_tokenize(question)

        filtered_question = [w for w in word_tokens if not w.lower() in stop_words]
        
        filtered_question = []
        
        for w in word_tokens:
            if w not in stop_words:
                filtered_question.append(w)
        
        question_relevance = get_question_relevance(story_words_relevance, filtered_question, story_unique_words)
        params["relevance"].append(question_relevance)
        # processed question stored in filtered_question
    
    params["open_ended"] = get_open_ended_nature()
    return params


In [7]:
def get_unprocessed_questions():
    questions_lst = []
    file = open("questions.txt", 'r')
    lines = file.readlines()
    for line in lines:
        questions_lst.append(line)
    file.close()
    return questions_lst

In [8]:
def test():
    [story_words_relevance, story_unique_words] = process_story()
    return process_questions(story_words_relevance, story_unique_words)
     
result = test()

In [9]:
re = result["relevance"]
op = result["open_ended"]

import numpy as np

questions = get_unprocessed_questions()   
rel = np.array(re)
ope = np.array(op)
rankings = {}
i = 0
unsorted_ranks = rel+ope
for rank in unsorted_ranks:
    rankings[questions[i]] = rank
    i += 1
# sorted_ranks = np.sort(rankings.keys())
# sorted_rankings = {}
# for rank in sorted_ranks:
#     sorted_ranks

In [13]:
# unsorted_ranks
sorted_ranks = dict(sorted(rankings.items(), key=lambda item: item[1]))

In [15]:
sorted_questions = []
for q in (sorted_ranks.keys()):
    if sorted_ranks[q] >= 7:
        sorted_questions.append(q)
sorted_questions.reverse()

In [16]:
sorted_questions

['Analyze the context of the plot of the given story.\n',
 'Why does the house beat "Safe, Safe, Safe" in the short story "A Haunted House" by Virginia Woolf?\n',
 'A moment later the light had faded. Out in the garden then? But the trees spun darkness for a wandering beam of sun. So fine, so rare, coolly sunk beneath the surface the beam I sought always burnt behind the glass. Death was the glass; death was between us; coming to the woman first, hundreds of years ago, leaving the house, sealing all the windows; the rooms were darkened. He left it, left her, went North, went East, saw the stars turned in the Southern sky; sought the house, found it dropped beneath the Downs. "Safe, safe, safe," the pulse of the house beat gladly. "The Treasure yours."The wind roars up the avenue. Trees stoop and bend this way and that. Moonbeams splash and spill wildly in the rain. But the beam of the lamp falls straight from the window. The candle burns stiff and still. Wandering through the house, op